#### Import Relevant Libraries

In [21]:
import pandas as pd
import numpy as np
import folium
import matplotlib.cm as plt
import matplotlib.colors as colors
from sklearn.cluster import KMeans

#### Loading data from the previous task

In [2]:
df = pd.read_csv('longlat.csv')

In [3]:
df.drop(columns='Unnamed: 0', inplace=True)

In [4]:
df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


#### Selecting Boroughs with the name 'Toronto' 

In [5]:
clus = df[df['Borough'].str.contains('Toronto',regex=False)]

In [6]:
clus.shape

(39, 5)

In [7]:
clus.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


#### Visualising the Toronto Boroughs using folium

In [8]:
Tmap = folium.Map(location=[43.651070,-79.347015], zoom_start=12)

for lat,lng,bgh,ngh in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
    label = '{}, {}'.format(ngh, bgh)
    label = folium.Popup(label, parse_html = True)
    
    folium.CircleMarker([lat,lng], radius=5, popup=label, color='black', fill=True, fill_color='#3186cc', fill_opacity=0.7,
    parse_html=False).add_to(Tmap)
    
Tmap

In [9]:
cluster = clus.drop(columns = ['Postal Code', 'Borough', 'Neighborhood'])

kms = KMeans(n_clusters = 3, random_state=0).fit(cluster)
cls = kms.labels_

In [10]:
clus.insert(5, 'Cluster Labels', cls)

In [11]:
clus.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1


#### Map cluster

In [23]:
FinalMap = folium.Map(location = [43.651070,-79.347015], zoom_start = 12)
k = 3

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = plt.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lon, ngh, clr in zip(clus['Latitude'], clus['Longitude'], clus['Neighborhood'], clus['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(clr), parse_html=True)
    folium.CircleMarker( [lat, lon], radius=5, popup=label, color = rainbow[clr-1], fill=True, fill_color = rainbow[clr-1],
    fill_opacity=0.7).add_to(FinalMap)
       
FinalMap